In [1]:
import os

from dotenv import load_dotenv

load_dotenv()

# 读取LangSmith Key
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "lang-env"
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
GPT_GOD_API_KEY = os.getenv("GPT_GOD_API_KEY")

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)

search.invoke("what is the weather in suzhou")

[{'url': 'https://www.meteoprog.com/weather/Suzhou/month/june/',
  'content': 'Suzhou (China) weather in June 2024 ☀️ Accurate weather forecast for Suzhou in June ⛅ Detailed forecast By month Current temperature "near me" Weather news ⊳ Widget of weather ⊳ Water temperature | METEOPROG. Meteoprog.com. ... 24 June +29 °+24° 25 June +29° ...'},
 {'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'Suzhou', 'region': 'Anhui', 'country': 'China', 'lat': 33.64, 'lon': 116.98, 'tz_id': 'Asia/Shanghai', 'localtime_epoch': 1719191524, 'localtime': '2024-06-24 9:12'}, 'current': {'last_updated_epoch': 1719190800, 'last_updated': '2024-06-24 09:00', 'temp_c': 28.6, 'temp_f': 83.4, 'is_day': 1, 'condition': {'text': 'Cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/119.png', 'code': 1006}, 'wind_mph': 3.8, 'wind_kph': 6.1, 'wind_degree': 35, 'wind_dir': 'NE', 'pressure_mb': 1005.0, 'pressure_in': 29.69, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 71, 'cloud': 87

In [3]:
# 创建召回器
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
documents

[Document(page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}),
 Document(page_content="Skip to main contentGo to API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith‚ÄãPythonTypeScriptpip install 

In [5]:
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [6]:
retriever.invoke("how to upload a dataset")[0]

Document(page_content='description="A sample dataset in LangSmith.")client.create_examples(    inputs=[        {"postfix": "to LangSmith"},        {"postfix": "to Evaluations in LangSmith"},    ],    outputs=[        {"output": "Welcome to LangSmith"},        {"output": "Welcome to Evaluations in LangSmith"},    ],    dataset_id=dataset.id,)# Define your evaluatordef exact_match(run, example):    return {"score": run.outputs["output"] == example.outputs["output"]}experiment_results = evaluate(    lambda input: "Welcome " + input[\'postfix\'], # Your AI system goes here    data=dataset_name, # The data to predict and grade over    evaluators=[exact_match], # The evaluators to score the results    experiment_prefix="sample-experiment", # The name of the experiment    metadata={      "version": "1.0.0",      "revision_id": "beta"    },)import { Client, Run, Example } from "langsmith";import { evaluate } from "langsmith/evaluation";import { EvaluationResult } from "langsmith/evaluation";co

In [7]:
retriever.invoke("对你知道的内容进行中文摘要总结Smith的功能")[0]

Document(page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'})

In [8]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [9]:
tools = [search, retriever_tool]

In [10]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="glm-4-flash",
                   base_url="https://api.gptgod.online/v1/",
                   api_key=GPT_GOD_API_KEY
                   )

In [11]:
# 使用语言模型
from langchain_core.messages import HumanMessage
response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hi there! How can I assist you today?'